## Librerías Necesarias

In [1]:
##### VOZ ######
# Text-to-Speech Conversion
import pyttsx3 as tts
# Reconocimiento de Voz
import speech_recognition

##### CHATBOT ######
# Intenciones y funcionalidad de chatbot
from neuralintents import GenericAssistant
# Manejo general del sistema
import sys
# Traducir texto, resumir texto
from transformers import pipeline
# Manejo de vectores
import numpy as np
# Calcular similaridad
import spacy

#######FUENTES DE INFORMACION Y PROCESAMIENTO#######
# Modulo para extraer noticias de Google News
from gnewsclient import gnewsclient
# Acceder a contenido de internet
import requests
# Acceder al texto dentro del html
from bs4 import BeautifulSoup
# Convertir html a texto
import html2text
# Analizar expresiones regulares
import re

## Asistente de Voz

### Voces Disponibles 

In [2]:
# Inciar objeto text-to-speech
speaker = tts.init()
# Obtener voces disponibles del sistema
voices = speaker.getProperty('voices')
# Obtener voces en español
voices_esp = []
print('\nVoces disponibles en Español:\n')


for voice in voices:
    if str(voice).find('spanish')>0:
        print("Voice: %s" % voice.name)
        print(" - ID: %s" % voice.id)
        print(" - Languages: %s" % voice.languages)
        print(" - Gender: %s" % voice.gender)
        print(" - Age: %s" % voice.age)
        print("\n")
        voices_esp.append(voice)

# Elegir español (VARIA EN SISTEMA OPERATIVO)
eleccion = voices_esp[1].id
print("Eligiendo idioma:",eleccion)
speaker.setProperty("voice", eleccion)


Voces disponibles en Español:

Voice: spanish
 - ID: spanish
 - Languages: [b'\x05es']
 - Gender: male
 - Age: None


Voice: spanish-latin-am
 - ID: spanish-latin-am
 - Languages: [b'\x05es-la']
 - Gender: male
 - Age: None


Eligiendo idioma: spanish-latin-am


### Pruebas iniciales - Asistente en Acción

In [3]:
# Asistente hablando
speaker.say("Hola mundo, ¡estoy vivo!")
# Reproducir sonido y parar
speaker.runAndWait()

## Reconocimiento de Audio

In [4]:
# Iniciar reconocimiento de voz
recognizer=speech_recognition.Recognizer()

### Pruebas iniciales - Reconocimiento en Acción

In [26]:
# Abrir Micrófono
with speech_recognition.Microphone() as mic:
    # Reducir ruido ambiental
    recognizer.adjust_for_ambient_noise(mic,duration=0.2)
    # Escuchar audio
    audio=recognizer.listen(mic)
    # Reconocer audio
    ticker=recognizer.recognize_google(audio,language="es-CO")
    
# Visualizar audio en texto
print("Audio reconocido:", ticker)

Audio reconocido: Hola


## Pruebas De Pregunta-Respuesta

In [8]:
# Flag para parar acción
done = False
while not done:
    try:
        with speech_recognition.Microphone() as mic:
            
            recognizer.adjust_for_ambient_noise(mic,duration=0.2)
            audio=recognizer.listen(mic)
            ticker=recognizer.recognize_google(audio,language="es-CO")#item
            #print(1)
            # Poner transformer aquí

            speaker.say("Dijiste: " + ticker)
            speaker.runAndWait()
            # Pasar texto a minúscula
            ticker=ticker.lower()
            print("Dijiste:",ticker)

    # Maquina no entender
    except speech_recognition.UnknownValueError:
        recognizer=speech_recognition.Recognizer()
        # Pedir info de nuevo
        speaker.say("No entendi lo que dijiste, por favor podrias repetir?")
        speaker.runAndWait()
    # Desactivar acciones con palabra clave
    if ticker=='chao':
        # Frase de despedida
        speaker.say("Chao pescao. Nos vemos por ahí.")
        speaker.runAndWait()
        done=True

Dijiste: hola hay alguien ahí
Dijiste: qué te cuentas
Dijiste: chao


# Mapeo

In [9]:
def hola():
    speaker.say("Hola, en que puedo ayudarte?")
    speaker.runAndWait()

In [10]:
def salir():
    speaker.say("Chao pescao. Nos vemos por ahí.")
    speaker.runAndWait()
    sys.exit(0)

In [19]:
def noticia():
    from gnewsclient import gnewsclient
    speaker.say("Claro que sí. Encontré esto:")
    speaker.runAndWait()
    # declare a NewsClient object 
    client = gnewsclient.NewsClient(language='spanish', location='Colombia', topic='Tecnología', max_results=5)

    # get news feed
    noticias = client.get_news()
    for i in range(len(noticias)):
        print(noticias[i]['title'])
        speaker.say(noticias[i]['title'])
        speaker.runAndWait()
        print(noticias[i]['link'])
    speaker.say("¿Cuál te gustaría escuchar?")
    speaker.runAndWait()

In [21]:
def resumen():
    import numpy as np
    speaker.say("Okey. Qué perezoso!, aquí tienes:")
    speaker.runAndWait()
    # declare a NewsClient object 
    with open('resumen_es.txt') as f:
        lines = f.readlines()
    print(lines)
    speaker.say(lines)
    speaker.runAndWait()

In [22]:
# Mappings pregunta-respuesta
mappings={
    "inicio":hola,
    "salir": salir,
    "noticias":noticia,
    "resumen":resumen
}

In [23]:
assistant=GenericAssistant('palabras.json',intent_methods=mappings)
assistant.train_model()

/home/alvaro/miniconda3/envs/ia/lib/python3.8/site-packages/neuralintents/main.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)
/home/alvaro/miniconda3/envs/ia/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.6414 - accuracy: 0.0638
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.5400 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5066 - accuracy: 0.2128
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2517 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1763 - accuracy: 0.3191
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0845 - accuracy: 0.3191
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9749 - accuracy: 0.3404
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7918 - accuracy: 0.4468
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8634 - accuracy: 0.3830
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5687 - accuracy: 0.5106
Epoch 11/

In [ ]:
while True:
    try:
        with speech_recognition.Microphone() as mic:
            recognizer.adjust_for_ambient_noise(mic,duration=0.2)
            audio=recognizer.listen(mic)
            message=recognizer.recognize_google(audio,language="es-CO")
            message=message.lower()
        assistant.request(message)
    except speech_recognition.UnknownValueError:
        recognizer=speech_recognition.Recognizer()

## Noticias 

In [50]:
# declare a NewsClient object 
client = gnewsclient.NewsClient(language='spanish', location='Colombia', topic='Tecnología', max_results=5)
  
# get news feed
noticias = client.get_news()
noticias

[{'title': 'Whatsapp permitirá convertir fotos en stickers en la misma app - El Tiempo',
  'link': 'https://news.google.com/__i/rss/rd/articles/CBMiVGh0dHBzOi8vd3d3LmVsdGllbXBvLmNvbS90ZWNub3NmZXJhL3doYXRzYXBwLWNvbW8tY29udmVydGlyLWZvdG9zLWVuLXN0aWNrZXJzLTYxODg4NNIBAA?oc=5',
  'media': None},
 {'title': 'Padres de familia del Liceo Francés hacen llamado a Policía por inseguridad - El Tiempo',
  'link': 'https://news.google.com/__i/rss/rd/articles/CBMibGh0dHBzOi8vd3d3LmVsdGllbXBvLmNvbS9ib2dvdGEvcm9ib3MtZW4tYm9nb3RhLXBhZHJlcy1kZWwtbGljZW8tZnJhbmNlcy1waWRlbi1zZWd1cmlkYWQtYS1sYS1wb2xpY2lhLTYxODYwNdIBAA?oc=5',
  'media': None},
 {'title': 'Samsung y Xiaomi destrozan a Apple por las "nuevas" características en el iPhone 13 - Pulzo.com',
  'link': 'https://news.google.com/__i/rss/rd/articles/CBMia2h0dHBzOi8vd3d3LnB1bHpvLmNvbS90ZWNub2xvZ2lhL3NhbXN1bmcteGlvYW1pLWJ1cmxhbi1hcHBsZS1wb3ItbnVldmFzLWNhcmFjdGVyaXN0aWNhcy1pcGhvbmUtMTMtUFAxMDc3NDY20gFvaHR0cHM6Ly93d3cucHVsem8uY29tL2FtcC90ZWNub2xvZ2lhL3NhbX

In [51]:
noticias[0]

{'title': 'Whatsapp permitirá convertir fotos en stickers en la misma app - El Tiempo',
 'link': 'https://news.google.com/__i/rss/rd/articles/CBMiVGh0dHBzOi8vd3d3LmVsdGllbXBvLmNvbS90ZWNub3NmZXJhL3doYXRzYXBwLWNvbW8tY29udmVydGlyLWZvdG9zLWVuLXN0aWNrZXJzLTYxODg4NNIBAA?oc=5',
 'media': None}

In [52]:
speaker.say(noticias[1]['title'])
speaker.runAndWait()

In [53]:
noticia_elegida = 0
url = noticias[noticia_elegida]['link']
res = requests.get(url)
html_page = res.content

soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all('p')
#text

In [54]:
texto = [texts for texts in list(text) if len(texts)>4]

In [55]:
texto_limpio = html2text.html2text(''.join(str(texto[2:])))
#print(texto_limpio)
texto_limpio = re.sub(r'\([^)]*\)',' ',texto_limpio)
texto_limpio = re.sub(r'\n','  ',texto_limpio)
texto_limpio = re.sub(r'\*',' ',texto_limpio)
print(texto_limpio)
#speaker.say(texto_limpio)
#speaker.runAndWait()

[       Whatsapp   permitirá convertir fotos en   stickers   sin necesidad de usar  aplicaciones externas o de terceros. Así se puede ver en las especificaciones  de prueba de versiones venideras que los desarrolladores pueden revisar  previamente.        Como siempre el portal WaBeta Info revela con antelación las funciones que  pronto se liberarán para   Whatsapp   , siendo esta una de las mas  interesantes y curiosas y que resuelve una constante petición de los usuarios.    ,      En contexto:   [  Las ocasiones en que WhatsApp revisa los mensajes de sus  usuarios  ]         Usted solo tendrá que tocar una fotografía de manera sostenida para que se  active un botón de stickers. Dicha foto puede ser seleccionada desde su  carrete de imágenes del celular. Al tocar el botón de stickers, esa foto  quedará automáticamente convertida en uno que podrá compartir de inmediato.          Interesante:   [  ¿Cómo bloquear su WhatsApp para que se abra solo con su  huella o Face ID?  ]         Tam

In [56]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

In [57]:
from sentence_utils import split_into_sentences
frase = split_into_sentences(texto_limpio)
frase

['[       Whatsapp   permitirá convertir fotos en   stickers   sin necesidad de usar  aplicaciones externas o de terceros.',
 'Así se puede ver en las especificaciones  de prueba de versiones venideras que los desarrolladores pueden revisar  previamente.',
 'Como siempre el portal WaBeta Info revela con antelación las funciones que  pronto se liberarán para   Whatsapp   , siendo esta una de las mas  interesantes y curiosas y que resuelve una constante petición de los usuarios.',
 ',      En contexto:   [  Las ocasiones en que WhatsApp revisa los mensajes de sus  usuarios  ]         Usted solo tendrá que tocar una fotografía de manera sostenida para que se  active un botón de stickers.',
 'Dicha foto puede ser seleccionada desde su  carrete de imágenes del celular.',
 'Al tocar el botón de stickers, esa foto  quedará automáticamente convertida en uno que podrá compartir de inmediato.',
 'Interesante:   [',
 '¿Cómo bloquear su WhatsApp para que se abra solo con su  huella o Face ID?',
 '

In [58]:
sentence = translator(frase)

In [59]:
sentence

[{'translation_text': '[Whatsapp will allow you to convert photos into stickers without using external or third-party applications.'},
 {'translation_text': 'So you can see in the test specifications of upcoming versions that developers can review in advance.'},
 {'translation_text': 'As always the WaBeta Info portal reveals in advance the functions that will soon be released for Whatsapp, being one of the most interesting and curious and that solves a constant request of users.'},
 {'translation_text': ', In context: [The occasions WhatsApp reviews the messages of its users ] You will only have to touch a photograph in a sustained way to activate a sticker button.'},
 {'translation_text': 'This photo can be selected from your cell phone image reel.'},
 {'translation_text': 'When you touch the sticker button, that photo will automatically become one that you can share immediately.'},
 {'translation_text': 'Interesting: ['},
 {'translation_text': 'How to block your WhatsApp so it opens 

In [60]:
todo_texto = [sentences['translation_text'] for sentences in sentence]
todo_texto = ' '.join(todo_texto)
todo_texto

"[Whatsapp will allow you to convert photos into stickers without using external or third-party applications. So you can see in the test specifications of upcoming versions that developers can review in advance. As always the WaBeta Info portal reveals in advance the functions that will soon be released for Whatsapp, being one of the most interesting and curious and that solves a constant request of users. , In context: [The occasions WhatsApp reviews the messages of its users ] You will only have to touch a photograph in a sustained way to activate a sticker button. This photo can be selected from your cell phone image reel. When you touch the sticker button, that photo will automatically become one that you can share immediately. Interesting: [ How to block your WhatsApp so it opens only with your fingerprint or Face ID? ] You may also store that sticker, as well as the person or persons in the group that receives it, in your favorite Whatsapp stickers tab, as explained in the [WaBet

In [63]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [64]:
resumen = summarizer(todo_texto)

In [65]:
resumen_en = resumen[0]['summary_text']
resumen_en

' WaBeta Info portal reveals in advance the functions that will soon be released for Whatsapp . You will only have to touch a photograph in a sustained way to activate a sticker button . This photo can be selected from your cell phone image reel . When you touch the sticker button, that photo will automatically become one that you can share immediately .'

In [66]:
translator_en_es = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")

In [67]:
resumen_es_list = translator_en_es(resumen_en)

In [68]:
resumen_es = resumen_es_list[0]['translation_text']
resumen_es

'El portal WaBeta Info revela de antemano las funciones que pronto serán lanzadas para Whatsapp . Sólo tendrá que tocar una fotografía de forma sostenida para activar un botón de pegatina . Esta foto se puede seleccionar desde el carrete de la imagen del teléfono celular . Cuando toque el botón de pegatina , esa foto se convertirá automáticamente en una que se puede compartir inmediatamente .'

In [69]:
speaker.say(resumen_es)
speaker.runAndWait()

In [71]:
noticias_encabezados = [noticia['title'] for noticia in noticias]
noticias_encabezados = [re.sub(' -\s.*\S$','',noticias_encabezado) for noticias_encabezado in noticias_encabezados]
noticias_encabezados

['Whatsapp permitirá convertir fotos en stickers en la misma app',
 'Padres de familia del Liceo Francés hacen llamado a Policía por inseguridad',
 'Samsung y Xiaomi destrozan a Apple por las "nuevas" características en el iPhone 13',
 'Cómo evitar que apaguen y cómo rastrear su celular Android en caso de robo',
 'Países donde sale más barato (o más caro) comprar el nuevo iPhone 13']

In [ ]:
!python -m spacy download es_core_news_lg

In [72]:
nlp = spacy.load('es_core_news_lg')

In [74]:
texto_comparar = nlp('policia')

[nlp(encabezado).similarity(texto_comparar) for encabezado in noticias_encabezados]

[0.12374630824919926,
 0.3852905161744063,
 0.08033732324755001,
 0.23713419326636698,
 -0.10259521986963996]

In [153]:
try:
    with speech_recognition.Microphone() as mic:
        recognizer.adjust_for_ambient_noise(mic,duration=0.2)
        audio=recognizer.listen(mic)
        ticker=recognizer.recognize_google(audio,language="es-CO")#item
        similitud = [nlp(encabezado).similarity(nlp(ticker)) for encabezado in noticias_encabezados]
        cual_not = noticias_encabezados[np.argmax(np.array(similitud))]
    
except speech_recognition.UnknownValueError:
    recognizer=speech_recognition.Recognizer()
    speaker.say("No entendi lo que dijiste, por favor podrias repetir?")
    speaker.runAndWait()

print(cual_not)
print(ticker)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
No caiga con el nuevo emoji de labios mordidos en WhatsApp: tiene 2 significados opuestos
emoji
